<a href="https://colab.research.google.com/github/bthcong2002/AI/blob/main/%C4%90%E1%BB%93_%C3%81n1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install names #package for random names

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 789 kB 8.8 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=e5bba5e414a8828ad9439f89bea1a5ce24253b076c97366c9e21c8494c29ef7b
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names


In [2]:
import numpy as np


class Schedule:

    def __init__(self, courseId, classId, teacherId):
  
        self.courseId = courseId
        self.classId = classId
        self.teacherId = teacherId

        self.roomId = 0
        self.weekDay = 0
        self.slot = 0

    def random_init(self, roomRange):
  
        self.roomId = np.random.randint(1, roomRange + 1, 1)[0]
        self.weekDay = np.random.randint(1, 6, 1)[0]
        self.slot = np.random.randint(1, 6, 1)[0]




In [3]:
def schedule_cost(population, elite):

    conflicts = []
    n = len(population[0])

    for p in population:
        conflict = 0
        for i in range(0, n - 1):
            for j in range(i + 1, n):

                if p[i].roomId == p[j].roomId and p[i].weekDay == p[j].weekDay and p[i].slot == p[j].slot:
                    conflict += 1
 
                if p[i].classId == p[j].classId and p[i].weekDay == p[j].weekDay and p[i].slot == p[j].slot:
                    conflict += 1

                if p[i].teacherId == p[j].teacherId and p[i].weekDay == p[j].weekDay and p[i].slot == p[j].slot:
                    conflict += 1

                if p[i].classId == p[j].classId and p[i].courseId == p[j].courseId and p[i].weekDay == p[j].weekDay:
                    conflict += 1

        conflicts.append(conflict)

    index = np.array(conflicts).argsort()

    return index[: elite], conflicts[index[0]]

In [4]:
import copy
import numpy as np


class GeneticOptimize:

    def __init__(self, popsize=30, mutprob=0.3, elite=5, maxiter=100):

        self.popsize = popsize
        self.mutprob = mutprob
        self.elite = elite
        self.maxiter = maxiter

# Khởi tạo quần thể
    def init_population(self, schedules, roomRange):

        self.population = []

        for i in range(self.popsize):
            entity = []

            for s in schedules:
                s.random_init(roomRange)
                entity.append(copy.deepcopy(s))

            self.population.append(entity)
# Đột biến với tỷ lệ 0.33
    def mutate(self, eiltePopulation, roomRange):

        e = np.random.randint(0, self.elite, 1)[0]
        pos = np.random.randint(0, 2, 1)[0]

        ep = copy.deepcopy(eiltePopulation[e])
    
        for p in ep:
            pos = np.random.randint(0, 3, 1)[0]
            operation = np.random.rand()

            if pos == 0:
                p.roomId = self.addSub(p.roomId, operation, roomRange)
            if pos == 1:
                p.weekDay = self.addSub(p.weekDay, operation, 5)
            if pos == 2:
                p.slot = self.addSub(p.slot, operation, 5)

        return ep

    def addSub(self, value, op, valueRange):

        if op > 0.5:
            if value < valueRange:
                value += 1
            else:
                value -= 1
        else:
            if value - 1 > 0:
                value -= 1
            else:
                value += 1

        return value
# Lai chéo
    def crossover(self, eiltePopulation):

        e1 = np.random.randint(0, self.elite, 1)[0]
        e2 = np.random.randint(0, self.elite, 1)[0]

        pos = np.random.randint(0, 2, 1)[0]

        ep1 = copy.deepcopy(eiltePopulation[e1])
        ep2 = eiltePopulation[e2]

        for p1, p2 in zip(ep1, ep2):
            if pos == 0:
                p1.weekDay = p2.weekDay
                p1.slot = p2.slot
            if pos == 1:
                p1.roomId = p2.roomId

        return ep1
#Tiến hóa
    def evolution(self, schedules, roomRange):

        bestScore = 0
        bestSchedule = None

        self.init_population(schedules, roomRange)

        for i in range(self.maxiter):
            eliteIndex, bestScore = schedule_cost(self.population, self.elite)

            if bestScore == 0:
                print('Solution found in loop {}'.format(i+1))
                bestSchedule = self.population[eliteIndex[0]]
                break

            newPopulation = [self.population[index] for index in eliteIndex]

            while len(newPopulation) < self.popsize:
                if np.random.rand() < self.mutprob:
                    newp = self.mutate(newPopulation, roomRange)
                else:
                    newp = self.crossover(newPopulation)

                newPopulation.append(newp)

            self.population = newPopulation

        return bestSchedule

In [5]:
import names
import csv
import numpy as np
def get_teacher(n):
  x=[]
  for i in range(n):
    x.append(names.get_first_name())
  return x
def get_data(schedules):
  subject = {'Math':get_teacher(4),'Physics':get_teacher(3),'Chemistry':get_teacher(3),'History':get_teacher(2),'Geography':get_teacher(2),'Literature':get_teacher(2)}
  classes = [101,102,103,104]

  for i in classes:
    for key,value in zip(subject.keys(), subject.values()):
      n = len(value)
      ran = np.random.randint(0,n)
      schedules.append(Schedule(key,i,value[ran]))

In [6]:
import prettytable 
import numpy as np

def vis(schedule):

    weekday = ['MON', 'TUE', 'WED', 'THU','FRI']
    col_labels = ['Weekday/Period', '1', '2', '3', '4','5','6']
    table_vals = [[i, '', '', '', '','',''] for i in weekday]

    table = prettytable.PrettyTable(col_labels, hrules=prettytable.ALL)

    for s in schedule:
        weekDay = s.weekDay
        slot = s.slot
        text = 'course: {} \n class: {} \n room: {} \n teacher: {}'.format(s.courseId, s.classId, s.roomId, s.teacherId)
        table_vals[weekDay - 1][slot] = text

    for row in table_vals:
        table.add_row(row)

    print(table)


if __name__ == '__main__':
    schedules = []

    # add schedule
    get_data(schedules)

    # optimization
    ga = GeneticOptimize(popsize=50, elite=10, maxiter=500)
    res = ga.evolution(schedules, 3)
    
    # visualization  
    vis_res = []
    for r in res:
      vis_res.append(r)
    vis(vis_res)


Solution found in loop 4
+----------------+---------------------+--------------------+---------------------+-------------------+---------------------+---+
| Weekday/Period |          1          |         2          |          3          |         4         |          5          | 6 |
+----------------+---------------------+--------------------+---------------------+-------------------+---------------------+---+
|      MON       |                     |  course: History   |  course: Chemistry  |   course: Math    |   course: History   |   |
|                |                     |     class: 101     |      class: 103     |     class: 102    |      class: 103     |   |
|                |                     |      room: 3       |       room: 3       |      room: 1      |       room: 1       |   |
|                |                     |    teacher: Owen   |     teacher: Lisa   |  teacher: Willard |     teacher: Owen   |   |
+----------------+---------------------+--------------------+----